### Load chunks as we did before

In [2]:
from chonkie import SemanticChunker
from pathlib import Path

md_filepath = Path("data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs.md")
md_txt = md_filepath.read_text()

chunker = SemanticChunker(
    embedding_model="minishlab/potion-base-8M",  # Default model
    threshold=0.5,                               # Similarity threshold (0-1) or (1-100) or "auto"
    chunk_size=2048,                              # Maximum tokens per chunk
    min_sentences=1                              # Initial sentences per chunk
)
chunk_texts = chunker.chunk(md_txt)

### Set up Weaviate Collection

In [79]:
from helpers import update_creds

AWS_ACCESS_KEY, AWS_SECRET_KEY, AWS_SESSION_TOKEN = update_creds()

In [ ]:
import weaviate

client = weaviate.connect_to_local(
    WEAVIATE_IP,
    headers = {
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SESSION_TOKEN,
    }
)

client.is_ready()

True

In [81]:
client.collections.delete("Chunks")

In [82]:
from weaviate.classes.config import Property, DataType, Configure, Tokenization

client.collections.create(
    name="Chunks",
    properties=[
        Property(
            name="document_title",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk_number",
            data_type=DataType.INT,
        ),
        Property(
            name="filename",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD
        ),
    ],
    vector_config=[
        Configure.Vectors.text2vec_aws(
            name="default",
            source_properties=["document_title", "chunk"],
            region="us-west-2",
            service="bedrock",
            model="amazon.titan-embed-text-v2:0"
        )
    ]
)

In [83]:
chunks = client.collections.use("Chunks")

### Import data

In [84]:
from tqdm import tqdm

with chunks.batch.fixed_size(batch_size=100) as batch:
    for i, chunk_text in tqdm(enumerate(chunk_texts)):
        obj = {
            "document_title": "HAI AI Index Report 2025",
            "filename": "data/pdfs/hai_ai-index-report-2025_chapter2_excerpts.pdf",
            "chunk": chunk_text.text,
            "chunk_number": i + 1,
        }

        # Add object to batch for import with (batch.add_object())
        # BEGIN_SOLUTION
        batch.add_object(
            properties=obj
        )
        # END_SOLUTION

33it [00:00, 20754.54it/s]


### RAG queries

How do we perform RAG in this scenario? 

This is a bit different, because we haven't embedded the images (or stored them in Weaviate).

In this scenario, let's:

- Retrieve text chunks
- Get images referred to in the text
- Convert the images to base64
- Send (retrieved text + images + prompt) to LLM for RAG

In [89]:
response = chunks.query.hybrid(
    query="Recent advances in RAG",
    limit=10
)

for o in response.objects:
    print(f"\n" + "=" * 40)
    print(o.properties["chunk"][:1000] + "...")


## Chapter 2: Technical Performance

## RAG: Retrieval Augment Generation (RAG)

An  increasingly  common  capability  being  tested  in  LLMs is retrieval-augmented generation (RAG). This approach integrates LLMs  with retrieval mechanisms  to enhance their  response  generation. The  model fi rst  retrieves  relevant information  from fi les  or  documents  and  then  generates  a response tailored to the user's query based on the retrieved content.  RAG  has  diverse  use  cases,  including  answering precise questions from large databases and addressing customer queries using information from company documents.

models. 2024 also saw the release of numerous benchmarks for  evaluating  RAG  systems,  including  Ragnarok  (a  RAG arena battleground) and CRAG (Comprehensive RAG benchmark). Additionally, specialized RAG benchmarks, such as FinanceBench for fi nancial question answering, have been developed to address speci fi c use cases.

## Berkeley Function Calling Leaderboard
...


In [90]:
import re

def extract_image_paths(text):
    """Extract image paths from markdown-style image references."""
    pattern = r'!\[.*?\]\((.*?)\)'
    return re.findall(pattern, text)

In [91]:
def get_image_base64s(image_paths, base_path=None):
    import base64
    base64_images = []
    for img_path in image_paths:
        full_path = Path(base_path) / img_path if base_path else Path(img_path)
        image_bytes = full_path.read_bytes()
        base64_string = base64.b64encode(image_bytes).decode("utf-8")
        base64_images.append(base64_string)

    return base64_images

In [92]:
all_chunks = ""
all_images = []

for o in response.objects:
    chunk_text = o.properties["chunk"]
    image_paths = extract_image_paths(chunk_text)
    print(f"Adding image paths: {image_paths}")
    all_images.extend(get_image_base64s(image_paths, base_path="data/parsed"))

    all_chunks += "\n\n" + chunk_text

Adding image paths: []
Adding image paths: []
Adding image paths: []
Adding image paths: []
Adding image paths: ['data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs_artifacts/image_000007_3921e74bcd236b403bab6328e8a11a403024116e502a0d0bbf5264f43935b867.png', 'data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs_artifacts/image_000008_33bf09ad90ca2e078ea69ec3e2e6beeea2fb2395ebd182a5b6e68c8d215648de.png']
Adding image paths: []
Adding image paths: ['data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs_artifacts/image_000009_8670c86bc1f4a1d41097f5ec286999521dc5bbf65615ebf4fb76a6a03b679acd.png']
Adding image paths: ['data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs_artifacts/image_000012_90c7c87f35758c0ce8c4a95f97089830098242f6ae4df85ab58cd0aa4f31fa8b.png']
Adding image paths: ['data/parsed/hai_ai-index-report-2025_chapter2_excerpts-parsed-w-imgs_artifacts/image_000006_60957158f1c4e3002ec6ab1ce22a95dd9fb64fde5d2273cea568

In [ ]:
message_list = [
    {
        "role": "user",
        "content": []
    }
]

for img in all_images:
    content = {
        "image": {
            "format": "png",
            "source": {
                "bytes": img
            },
        }
    }
    message_list[0]["content"].append(content)

task_text = """
What does this text tell us about the latest developments in RAG?

Describe the details from the figures as well, if necessary.
""" + "\n\n" + all_chunks
message_list[0]["content"].append({"text": task_text})

In [ ]:
import boto3
import json

client = boto3.client(
    "bedrock-runtime",
    region_name="us-west-2",
)

# MODEL_ID = "us.amazon.nova-lite-v1:0"
MODEL_ID = "us.amazon.nova-pro-v1:0"

# Define your system prompt(s).
system_list = [    {
        "text": "You are an expert. Read the provided text and content of these images and answer the questions thoughtfully but succinctly if possible."
    }
]

# Configure the inference parameters.
inf_params = {"maxTokens": 300, "topP": 0.1, "topK": 20, "temperature": 0.3}

native_request = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}
# Invoke the model and extract the response body.
response = client.invoke_model(modelId=MODEL_ID, body=json.dumps(native_request))
model_response = json.loads(response["body"].read())

# Print the text content for easy readability.
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)

/opt/conda/lib/python3.12/site-packages/botocore/auth.py:425: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow()



[Response Content Text]
### Latest Developments in RAG

The text highlights several key advancements and benchmarks in Retrieval-Augmented Generation (RAG):

1. **RAG Systems**: RAG integrates LLMs with retrieval mechanisms to enhance response generation. It retrieves relevant information from files or documents and generates tailored responses based on the retrieved content. Use cases include answering precise questions from large databases and addressing customer queries using company documents.

2. **Benchmarks**: 
   - **Ragnarok**: A RAG arena battleground.
   - **CRAG**: Comprehensive RAG benchmark.
   - **FinanceBench**: Specialized RAG benchmark for financial question answering.

3. **Technical Innovations**:
   - **Anthropic's Contextual Retrieval**: Introduced in September 2024, significantly enhances RAG's retrieval capabilities.

### Figures and Details

1. **Berkeley Function Calling Leaderboard**:
   - Evaluates LLMs' ability to accurately call functions or tools.
   - I

In [ ]:
client.close()